# WINDNODE ABW - Multi Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__authors__ 		= "[Guido Pleßmann](https://github.com/gplssm), [Jonathan Amme](https://github.com/nesnoj), [Julian Endres](https://github.com/nailend), " <br>

<div class="alert alert-warning">
<font size="4">We apologise for the fact that the plots in the notebooks are only shown as static graphics due to limited ressources. Executing the notebook locally (using the results 
<a href="https://zenodo.org/">package from zenodo</a>
) enables the interactive plots and will provide more precise information.</font>


</div>

## Intro

This jupyter notebook provides plots and information to compare the results of the dispatch-optimization of __all__ [scenarios](https://windnode-abw.readthedocs.io/en/latest/scenarios.html) by the study [__"A regional energy system model for Anhalt-Bitterfeld-Wittenberg"__](https://windnode-abw.readthedocs.io/en/latest/). The different scenarios cover various combinations of renewable energy penetration, area restrictions and flexibility options in heat and power sector. The notebooks will, therefore, give an overall view of energy supply and demand by the various scenarios and an insight into scenario-specific distribution and flexiblity effects.
    

  **The representation in jupyter notebooks is intended to ensure transparency and to provide a low entry barrier for further analysis.* 

<div class="alert alert-info">
<b>Notes on plots</b>


<ol>
<li> Some plots are generated with plotly and may not show up initially as Javascript is not enabled by default.</li>
<li> This can be solved by clicking <b>File -> "Trust Notebook"</b>.</li>
</ol> 
    
<b>These plots have interactive features:</b>

<ul>
<li> the web-representation might only show static graphics </li>
<li> hovering over the plot will display additional infos </li>
<li> clicking the legend selects data </li>
</ul>
    
    
</div>

# Table of Contents

* [Scenario information](#scenario_information)
* [0 Scenario_overview](#0_Scenario_overview)
* [1 Area required by RES](#1_area_required_by_res)
* [2 Demand and Generation](#2_demand_and_generation)
* [3 Energy Exchange and Autarky](#3_exchange_autarky)
* [4 Energy Mix](#4_energy_mix)
* [5 Emissions](#5_emissions)
* [6 Costs](#6_costs)
* [7 Power Grid](#7_power_grid)
* [8 Flexibility](#8_flexibility)

In [ ]:
######## WINDNODE ###########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()
# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')
# import scripts
from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######## DATA ###########
import re
import pandas as pd
from shapely import wkt

######## PLOTTING ###########
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()
# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Scenario information<a class="anchor" id="scenario_information"></a>

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=scenarios,
                                      force_new_results=force_new_results)

# 0 Scenario_overview <a class="anchor" id="0_Scenario_overview"></a>
## Key results

The following plots provide the key results of all scenario:


1. Costs and specific emissions

    - Total Costs: Total costs for heat and electricity supply
    - LCOE: Levelized Costs Of Electricity [calculation LCOE](#6_2_LCOE/H)
    - LCOH: Levelized Costs Of Heat [calculation LCOH](#6_2_LCOE/H)
    - Specific Emissions: Electrical and Heat Emissions per generated electricity
    
    
2. Area restrictions and eletricity exchange

    - RES Area Wind (VR/EG): Relative area used for Wind Energy compared to legal StatusQuo [more info](#2_area_required_by_res)
    - RES Area PV ground: Relative area used for PV ground compared to StatusQuo [more info](#2_area_required_by_res)
    - Intra-reg. Exchange: Accumulated amount of energy exchanged between the municipalities
    - El. Exports: Accumulated electricity exports to the national grid 
    
    
3. Autarky, Storage and DSM

    - Autarky:  Ratio of electricity supply and demand [calculation of Autarky](#8_flexibility)
    - El. Storage Use: Utilization Rate for electrical Storages [calculation of Utilization Rate (Storage)](#8_flexibility)
    - Heat Storage Use: Utilization Rate for heat Storages [calculation of Utilization Rate (Storage)](#8_flexibility)
    - DSM Utilization Rate: Utilization Rate for Demand-Side-Management [calculation of Utilization Rate (DSM)](#8_flexibility)
    
_*[This function](https://github.com/windnode/WindNODE_ABW/blob/7e6062e1b9be5f4c1aaa31de6a8fff4b14c995ba/windnode_abw/tools/draw.py#L942-L1102) is used for plotting._

In [ ]:
scenario_results = plot_key_scenario_results(results_scns, scenarios, scenario_order=scenario_order)

# 1 Area required by RES <a class="anchor" id="1_area_required_by_res"></a>

## 1.1  Land-use restrictions and Power Potential in RE Scenarios

The following plot shows the maximum RES potential for each land-use scenario and technology. Various restrictions can be applied. These land-use scenarios set an upper-limit to the different basic scenarios (NEP/ISE).

<div class="alert alert-block alert-info">

<b>Technology-specific restrictions of land use scenarios:</b>
<ul>
<li><b>Wind</b>:</li>
    <ul>
    <li>Distance to settlements <i>(500m/1000m)</i></li>
    <li>use of forests <i>(with: w / without: wo)</i></li>
    <li>percentage of available area due to restrictions resulting from case-by-case decisions <i>(10%)</i></li>
    </ul>

<li> <b>PV ground</b>:</li>
    <ul>
    <li>Restrictions that apply (hard: H / hard+soft: HS)</li>
    <li>percentage of total available agricultural area as upper limit <i>(0.1% / 1%)</i></li>
    </ul>
    <li> <b>PV rooftop</b>:</li>
        <ul>
        <li>Percentage of total potential <i>(50% / 100%)</i></li>
        </ul>
</ul>
    
    
Note: The specific land use (ha/MWp) of PV are different for 2035 and 2050. For the sake of simplicity, the value of 2050 is used in this plot (cf. assumptions).

</div>

Absolute values are represented in <b>colored</b> bars and relative (always relative to the max. installable power for the regulatory status quo) in  <b>grey</b> bars. The regulatory status quo is highlighted in <b>bold</b> for each technology.

_*[This function](https://github.com/windnode/WindNODE_ABW/blob/7e6062e1b9be5f4c1aaa31de6a8fff4b14c995ba/windnode_abw/tools/draw.py#L1279-L1360) is used for plotting._

In [ ]:
power_pot_land_use(regions_scns, scenarios)

Further on, the following plot shows the maximum installable power for each RE scenario and technology. The land-use scenarios are therefore applied to the basic scenarios <b>NEP</b> and <b>ISE</b>. 

<div class="alert alert-block alert-info">

<b> RES scenario convertion: </b>
<ul>
<li><b>RE - </b>:</li>
    <ul>
    <li> Wind capacity limited to designated areas for wind energy (VR/EG).</li>
    <li>Areas along highways and railway tracks and 0.1 % of agricultural land (strict and weak restrictions apply) are used for PV ground.</li>
    <li>No restrictions for rooftop PV. </li>
    </ul>
    
<li><b>RE </b>:</li>
    <ul>
    <li>no restrictions on installable capacity for any RE technology due to available areas. <i>(RE not mentioned in scenario name)</i> </li>
    </ul>
    
<li><b>WIND + </b>: <i>(only NEP)</i></li>
        <ul>
        <li>Minimum distance to settlements reduced to 500 m and wind installations in forests not allowed.</li>
        <li>No restrictions for both PV installations.  </li>
        </ul>
        
<li><b>PV + </b>: <i>(only NEP)</i></li>
    <ul>
    <li>No restrictions for wind energy.</li>
    <li>Areas along highways and railway tracks and 1.0 % of agricultural land (strict and weak restrictions apply) are used for PV ground.</li>
    <li>50 % of PV rooftop potential</li>
    </ul>
    
<li><b>RE ++ </b>:</li>
    <ul>
    <li>Minimum distance of wind turbines to settlements reduced to 500 m and wind installations in forests allowed.</li>
    <li>Areas along highways and railway tracks and max 1.0 % of agricultural land (only strict restrictions).</li>
    <li>100% of PV rooftop potential </li>
    </ul>
</ul>
    
    
<i>*if no restrictions, capacities according to NEP / ISE are used </i>
</div>

<b>Absolute</b> values are represented in <b>colored</b> bars and <b>relative</b> values (relative to the max. installable power for the regulatory status quo of 2017) in <b>thin black</b> bars.

_*[This function](https://github.com/windnode/WindNODE_ABW/blob/7e6062e1b9be5f4c1aaa31de6a8fff4b14c995ba/windnode_abw/tools/draw.py#L1363-L1479) is used for plotting._

In [ ]:
power_pot_scenarios(regions_scns, scenarios)

### 1.2 Available areas: Wind Energy

The following plots show the available area for wind energy systems after including respective land-use restrictions. A comprehensive description of how those areas are determined see [the documentation](https://windnode-abw.readthedocs.io/en/latest/land_availability.html#)

- The black spots mark 100% of the determined available area after land-use restrictions.
- In this study, we only use 10% of this area for wind turbines as this has proven to be a good approximation for individual case decisions in the region.
- The resulting amount is represented by the coloration.

In [ ]:
df_pot_area = pd.DataFrame({scn: area_ha.droplevel(level=1)
              for scn, area_ha
              in regions_scns['StatusQuo'].pot_areas_wec.area_ha.groupby(level=1)})

df_geoms = pd.DataFrame({scn: area_ha.droplevel(level=1)
              for scn, area_ha
              in regions_scns['StatusQuo'].pot_areas_wec.geom.groupby(level=1)})

df_pot_area.drop(columns='s1000f0', inplace=True)
df_geoms.drop(columns='s1000f0', inplace=True)

name_mapping = {'sq': 'Wind legal SQ',
                's1000f1': 'Wind 500m w forest 10-perc',
                's500f0': 'Wind 500m wo forest 10-perc',
                's500f1': 'Wind 500m w forest 10-perc'}

df_pot_area.rename(columns=name_mapping, inplace=True)
df_geoms.rename(columns=name_mapping, inplace=True)

fig, axes = plt.subplots(2, 2, figsize=(12,8))

for ax, (key, data) in  zip(axes.flat, df_pot_area.iteritems()):
    plot_geoplot(key, data, regions_scns['StatusQuo'], ax=ax,
                 unit='ha', cmap=cmap, vmin=0, vmax=14000)

for ax, (key, geoms) in  zip(axes.flat, df_geoms.iteritems()):
    
    gdf_geoms = geoms.rename('geom').to_frame().dropna()
    gdf_geoms = gdf_geoms['geom'].apply(wkt.loads)
    gdf_geoms = gdf_geoms.rename('geom').to_frame()
    gdf_geoms = gpd.GeoDataFrame(gdf_geoms, geometry='geom')
    gdf_geoms.plot(color='black', edgecolor='black', alpha=0.5, ax=ax)

fig.suptitle('Required Area',
    fontsize=16,
    fontweight='normal')
plt.tight_layout()
plt.show()

### 1.3 Available areas: Ground-Mounted PV

The following plots show the available area for PV ground systems per potential area type after including respective land-use restrictions. A comprehensive description of how those areas are determined see [the documentation](https://windnode-abw.readthedocs.io/en/latest/land_availability.html#)

- The black spots mark 100% of the determined available area after land-use restrictions.
- Depending on the scenario max 0.1% or 1% of the total agricultural area can be used for PV ground systems.
- The potential areas type are aggregated per land-use scenario. 
- The resulting amount is represented by the coloration.

In [ ]:
df_pot_area = pd.DataFrame({scn: area_ha.droplevel(level=1)
              for scn, area_ha
              in regions_scns['StatusQuo'].pot_areas_pv.area_ha.groupby(level=1)})

df_geoms = pd.DataFrame({scn: area_ha.droplevel(level=1)
              for scn, area_ha
              in regions_scns['StatusQuo'].pot_areas_pv.geom.groupby(level=1)})

name_mapping = {'agri_h': 'Agriculture hard', 'agri_hs': 'Agricultur hard+soft',
                'bab_h': 'Motorways hard', 'bab_hs': 'Motorways hard+soft',
                'rail_h': 'Railways hard', 'rail_hs': 'Railways hard+soft'}

df_pot_area.rename(columns=name_mapping, inplace=True)
df_geoms.rename(columns=name_mapping, inplace=True)

fig, axes = plt.subplots(3, 2, figsize=(12,10))

vmax = [8000,8000,70,70,300,300]
vmin = [0,0,0,0,0,0]
scale = list(zip(*[vmax, vmin]))

for ax, (key, data), (vmax, vmin) in  zip(axes.flat, df_pot_area.iteritems(), scale):
    plot_geoplot(key, data, regions_scns['StatusQuo'], ax=ax,
                 unit='ha', cmap=cmap, vmin=vmin, vmax=vmax)



for ax, (key, geoms) in  zip(axes.flat, df_geoms.iteritems()):
    
    gdf_geoms = geoms.rename('geom').to_frame().dropna()
    gdf_geoms = gdf_geoms['geom'].apply(wkt.loads)
    gdf_geoms = gdf_geoms.rename('geom').to_frame()
    gdf_geoms = gpd.GeoDataFrame(gdf_geoms, geometry='geom')
    gdf_geoms.plot(color='black', edgecolor='black', alpha=0.5, ax=ax)
    
fig.suptitle('Available Areas: Ground-Mounted',
     fontsize=16,
     fontweight='normal',
            y=1)
plt.tight_layout()
plt.show()

### 1.4 Available areas: Roof-Mounted PV
The following plots show the available area for PV roof-mounted systems. There is no specific information about the exact locations of the potential areas.

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
df_pot_area = regions_scns['StatusQuo'].pot_areas_pv_roof.sum(axis=1).rename('area')

plot_geoplot(None, df_pot_area, regions_scns['StatusQuo'], ax=ax,
             unit='ha', cmap=cmap)

fig.suptitle('PV Roof-Mounted',
            fontsize=16,
            y=1,
            fontweight='normal')
plt.tight_layout()
plt.show()

# 2 Demand and Generation <a class="anchor" id="2_demand_and_generation"></a>

## 2.1 Installed Capacities Electricity/Heat
The following figures show the total installed capacity for both electricity and heat in the ABW-region for each scenario.

In [ ]:
cap_el = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity electricity supply'].sum(axis=0)
                       for scn in scenarios}).T

cap_el = cap_el.assign(sum=cap_el.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1)

fig = go.Figure()

for tech, data in cap_el.iteritems():
    fig.add_trace(go.Bar(x=cap_el.index,
                         y=data,
                         name=PRINT_NAMES[tech],
                         marker=dict(color=COLORS[tech]),
                         hovertemplate='%{y:.1f} MW',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Installed Capacity Electricity Supply',
               y=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True,)

fig.show()

In [ ]:
cap_th = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity heat supply'].sum(axis=0)
                       for scn in scenarios}).T

cap_th = cap_th.assign(sum=cap_th.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1)

fig = go.Figure()
colors_el = [COLORS[c] for c in cap_th.columns]
for tech, data in cap_th.iteritems():
    fig.add_trace(go.Bar(x=cap_th.index,
                         y=data,
                         name=PRINT_NAMES[tech],
                         marker=dict(color=COLORS[tech]),
                         hovertemplate='%{y:.1f} MW',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Installed Capacity Heat Supply',
               y=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

## 2.2 Electricity and Heat Generation
The following figures show the annual total of genrated electricity or heat for the ABW-region for each scenario.

In [ ]:
df_gen_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Electricity generation']
                       for scn in scenarios}).T
df_gen_el = df_gen_el / 1e3 #
fig = go.Figure()
for tech, data in df_gen_el.iteritems():
    fig.add_trace(go.Bar(x=df_gen_el.index,
                         y=data,
                         name=tech,
                         marker=dict(color=COLORS_PRINT[tech]),
                         hovertemplate='%{y:.1f} GWh',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Electricity Generation',
               y=1),
    barmode='stack',
    height=800,
    hovermode="x unified",
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    yaxis=dict(title='GWh',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

In [ ]:
df_gen_heat = pd.DataFrame({scn: results_scns[scn]['results_t']['Heat generation']
                       for scn in scenarios}).T
df_gen_heat = df_gen_heat / 1e3 #
fig = go.Figure()
for tech, data in df_gen_heat.iteritems():
    fig.add_trace(go.Bar(x=df_gen_heat.index,
                         y=data,
                         name=tech,
                         marker=dict(color=COLORS_PRINT[tech]),
                         hovertemplate='%{y:.1f} GWh',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Heat Generation',
               y=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    yaxis=dict(title='GWh',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

## 2.3 Electricity and Heat Demand
The following figure shows the annual total demand of both electricity and heat for the ABW-region for each scenario.

In [ ]:
df_el = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Stromnachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Wärmenachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = df_heat.T.groupby([s.split('_')[0] for s in df_heat.columns.values]).sum().T

df_demand = pd.concat([df_el, df_heat], axis=0, keys=['Electricity', 'Heat'], sort=False)
df_demand = df_demand.rename(columns=PRINT_NAMES)
df_demand = df_demand / 1e3

fig = make_subplots(rows=len(scenarios), cols=1, shared_xaxes=True, vertical_spacing=0)
for row, (key, df) in enumerate(df_demand.groupby(level=1)):
    for i, (tech, data) in enumerate(df.iteritems()):
        fig.add_trace(go.Bar(y=list(zip(*data.index.swaplevel())),
                             x=data,
                             name=tech,
                             orientation='h',
                             marker_color=COLORS_PRINT[tech],
                             legendgroup=tech,
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=not bool(row)), row=row+1, col=1)

fig.update_layout(
    title='Demand',
    barmode='stack',
    hovermode="y unified",
    yaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    height=50*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_xaxes(title_text="GWH", row=len(scenarios), col=1, matches='x')
fig.show() 

# 3 Energy Exchange and Autarky <a class="anchor" id="3_exchange_autarky"></a>

## 3.1 Energy Exchange
The following plot show the relations of imported and exported electricity with the national grid.

In [ ]:
df_exchange = pd.DataFrame({scn:{'import' : results_scns[scn]['flows_txaxt']['Stromerzeugung']['import'].sum(),
                        'export': results_scns[scn]['flows_txaxt']['Stromnachfrage']['export'].sum()}
             for scn in scenarios}).T#.rename(columns=PRINT_NAMES)
df_exchange = df_exchange / 1e3 #GWh

fig = go.Figure()
fig.add_trace(go.Bar(x=df_exchange.index,
                    y=df_exchange['import'],
                    orientation='v',
                    name=PRINT_NAMES['import'],
                     marker_color=colors[0],
                     #opacity=0.5,
                        hovertemplate='%{y:.1f} GWh'))
fig.add_trace(go.Bar(x=df_exchange.index,
                        y=-df_exchange['export'],
                        orientation='v',
                        name=PRINT_NAMES['export'],
                        marker_color=colors[20],
                     #opacity=0.5,
                        hovertemplate='%{y:.1f} GWh'))
fig.update_layout(
    title='Import and Export (National Grid)',
    barmode='relative',
    autosize=True,
    hovermode="x unified",
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
 legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))
fig.update_yaxes(title='GWh', showspikes=False)
fig.show()

## 3.2 Autarky


Annual Autarky describes the ratio of the electricity gerneration to the demand within the region Anhalt-Bitterfeld-Wittenberg (ABW).

<div class="alert alert-block alert-info">
<b>Notes on Autarky calculation</b>

<ul>
<li> <b>(1) Annual Autarky (ABW):</b> degree of autark electricity supply for ABW disregarding dimension of time
  $$Autarky_{Annual,ABW,\%} = \frac{\sum_{t=1}^{8760} E_{supply,mun,t}}{\sum_{t=1}^{8760} E_{demand,mun,t}} \cdot 100\,\%$$ </li>
 </ul>  
  </div>  
A further perspective results from the percentage of hours in a year at which the electricity demand is entirely served by local supply.
 
<div class="alert alert-block alert-info">

<ul>
<li> <b> (2) Autark Hours (ABW):</b>
  $$Autark\,hours_{Annual,ABW,\%} = \frac{\sum_{t=1}^{8760} (\frac{E_{supply,ABW,t}}{E_{demand,ABW,t}} \geq 1)}{8760}   \cdot 100\,\%$$ </li>

</ul>
    
</div>

The electricity that exceeds the local demand can be distributed to the neighboring municipalities and thus contributes to the degree of Autarky of the entire region.

**_Only the electrical sector is include in the indicator "Autarky" as there are no transregional heating grids. However, parts of the electrical demand result from "Power-to-Heat" flexibility and is therefore included partially._

In [ ]:
df_autarky = pd.DataFrame({scn:{'Autarky' : float(results_scns[scn]['highlevel_results']['Autarky']),
                        'Autark Hours': float(results_scns[scn]['highlevel_results']['Autark hours'])}
             for scn in scenarios}).T

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,)

fig.add_trace(go.Bar(x=df_autarky.index,
                    y=df_autarky['Autarky'],
                    orientation='v',
                    name='Autarky',
                     marker_color=colors[0],
                        hovertemplate='%{y:.1f} %'),row=1, col=1, secondary_y=False)
fig.add_trace(go.Bar(x=df_autarky.index,
                        y=df_autarky['Autark Hours'],
                        orientation='v',
                        name='Autark Hours',
                        marker_color=colors[20],
                        hovertemplate='%{y:.1f} %'),row=2, col=1, secondary_y=False)
fig.update_layout(
    title='Autarky',
    barmode='group',
    autosize=True,
    hovermode="x unified",
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
 legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))
fig.update_yaxes(title='%', showspikes=False)
fig.show()

# 4 Energy Mix <a class="anchor" id="4_energy_mix"></a>

### 4.1 Power Generation and Demand
The following plot shows the accumulated annual electricity generation and demand of the ABW-Region.

In [ ]:
df_generation = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Stromerzeugung'].sum()
                       for scn in scenarios}).T
df_demand = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Stromnachfrage'].sum()
                       for scn in scenarios}).T

fig = go.Figure()
for tech, data in df_generation.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=data / 1e3,
                            orientation='v',
                            name=PRINT_NAMES[tech],
                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh'))

for tech, data in df_demand.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=-data / 1e3,
                            orientation='v',
                            name=PRINT_NAMES[tech],
                            marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh',))

fig.update_layout(
    title=dict(text='Power Generation and Demand',
               y=1),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    barmode='relative',
    height=900,
    autosize=True,
    hovermode="x unified",
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
)
fig.update_yaxes(title='GWh', showspikes=False)
fig.show()

## 4.2 Heat Generation and Demand
The following plot shows the accumulated annual heat generation and demand of the ABW-Region.

In [ ]:
df_generation = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Wärmeerzeugung'].sum()
                       for scn in scenarios}).T#ename(columns=PRINT_NAMES)
df_demand = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Wärmenachfrage'].sum()
                       for scn in scenarios}).T#.rename(columns=PRINT_NAMES)
df_generation = df_generation / 1e3
df_demand = df_demand / 1e3

fig = go.Figure()
for tech, data in df_generation.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=data,
                            orientation='v',
                            name=PRINT_NAMES[tech],
                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh'))

for tech, data in df_demand.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=-data,
                            orientation='v',
                            name=PRINT_NAMES[tech],
                            marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh',))


fig.update_layout(
    title=dict(text='Heat Generation and Demand',
               y=1),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    barmode='relative',
     height=800,
    autosize=True,
    hovermode="x unified",
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),)
fig.update_yaxes(title='GWh', showspikes=False)
fig.show()

# 5 Emissions <a class="anchor" id="5_emissions"></a>

## 5.1 Emissions absolute
The following plot show the total accumulated CO2 emissions per sector/technology for each scenario.

In [ ]:
df_emissions = pd.DataFrame(
    {scn : pd.concat([pd.Series(data=results_scns[scn]['results_axlxt'][key].sum().sum(), index=[key]) \
                      for key in results_scns[scn]['results_axlxt'] if 'total' in key]) \
     for scn in scenarios}).T

df_emissions['CO2 emissions grid total'] = df_emissions[['CO2 emissions grid total',
                                                         'CO2 emissions grid new total']]
df_emissions = df_emissions.drop(columns='CO2 emissions grid new total')

fig = go.Figure()
for i, (tech, data) in enumerate(df_emissions.iteritems()):
    fig.add_trace(go.Bar(x=data.index,
                         y=data,
                         name=tech,
                         orientation='v',
                         marker_color=colors[5*i],
                         legendgroup=tech,
                         hovertemplate='%{y:.1f} t CO2',))
fig.update_layout(
    title='Emissions',
    barmode='stack',
    hovermode="x unified",
    height=600,
    autosize=True,
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
               tracegroupgap=0))

fig.update_yaxes(title_text="t CO2")
fig.show() 

# 6 Costs <a class="anchor" id="6_costs"></a>

## 6.1 Costs per Sector and Technology

The following plots show the accumulated total costs per technology for both heat and electricity sector. The total cost include fixed, variable cost as welll as CO2 certificat prices.

In [ ]:
df_cost_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Total costs electricity supply']
                       for scn in scenarios}).T#.rename(columns=PRINT_NAMES)
df_cost_el = df_cost_el/ 1e6 # to Million €

fig = go.Figure()

for tech, data in df_cost_el.iteritems():
    fig.add_trace(go.Bar(x=df_cost_el.index,
                         y=data,
                         name=PRINT_NAMES[tech],
                         marker=dict(color=COLORS[tech]),
                         hovertemplate='%{y:.1f} M €',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Total electrical costs distribution per technology', y=1),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis=dict(tickfont_size=12, categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    yaxis=dict(title='M €', titlefont_size=16, tickfont_size=14),
    autosize=True,)

fig.show()

In [ ]:
df_cost_th = pd.DataFrame({scn: results_scns[scn]['results_t']['Total costs heat supply']
                       for scn in scenarios}).T#.rename(columns=PRINT_NAMES)
df_cost_th = df_cost_th/ 1e6 # to Million €

fig = go.Figure()

for tech, data in df_cost_th.iteritems():
    fig.add_trace(go.Bar(x=df_cost_th.index,
                         y=data,
                         name=PRINT_NAMES[tech],
                         marker=dict(color=COLORS[tech]),
                         hovertemplate='%{y:.1f} M €',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Total thermal costs distribution per technology', y=1),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    xaxis=dict(tickfont_size=12, categoryorder='array',
              categoryarray=scenario_order,
              tickmode = 'linear',),
    yaxis=dict(title='M €', titlefont_size=16, tickfont_size=14),
    autosize=True,)
fig.show()

## 6.2 LCOE, LCOH and Renewable Share <a class="anchor" id="6_2_LCOE/H"></a>
The following plot shows the levelized cost of electricity and heat in comparisson to the RE share. The RE share is the ratio of total generated electricity of renewable energy and the total demand for each scenario.

<div class="alert alert-block alert-info">
<b>Notes on LCOE calculation</b>

- Total LCOE calculate as $LCOE=\frac{expenses_{el.total}}{demand_{el.,total}}$, likewise total LCOH calculate as $LCOH=\frac{expenses_{th.,total}}{demand_{th.,total}}$
- Total expenses $expenses_{el.total}$ are annual expenses. Investment costs are discounted to one year using equivalent periodic costs
</div>

In [ ]:
scenario_results_ext = scenario_results[1].join(
    scenario_results[2].drop("Scenario", axis=1)).join(
    scenario_results[3].drop("Scenario", axis=1)).copy()

tmp_dict = {
    "RES share [%]": [],
    "Autark hours [%]": [],
    'Heat Storage Use [GWh]': [],
    'El. Storage Use [GWh]': [],
    'Net DSM activation [GWh]': []
}
for scenario in scenarios:
    tmp_dict["RES share [%]"].append(results_scns[scenario]["results_t"]['Electricity generation'].drop(
        ["Large-scale CHP", "Open-cycle gas turbine", "Combined-cycle gas turbine"]).sum() / float(results_scns[scenario]["highlevel_results"]["Electricity demand total"]) * 100)
    tmp_dict["Autark hours [%]"].append(float(results_scns[scenario]["highlevel_results"]["Autark hours"]) * 100)
    tmp_dict['Heat Storage Use [GWh]'].append(results_scns[scenario]["results_axlxt"]['Wärmespeicher nach Gemeinde'].sum().sum() / 1e3)
    tmp_dict['El. Storage Use [GWh]'].append(results_scns[scenario]["results_axlxt"]['Batteriespeicher nach Gemeinde'].sum().sum() / 1e3)
    tmp_dict['Net DSM activation [GWh]'].append(float(results_scns[scenario]["highlevel_results"]['Net DSM activation']) / 1e3)
additional_key_figures = pd.DataFrame(tmp_dict, index=scenarios)
scenario_results_ext = scenario_results_ext.join(additional_key_figures, on="Scenario").set_index("Scenario").sort_values("RES share [%]")
scenario_results_ext = scenario_results_ext.drop("StatusQuo", axis=0)

fig = make_subplots(specs=[[{"secondary_y": True}]])

re_scn_matches = [re.search('((RE\+\+)|(RE\+)|(WIND\+)|(PV\+)|(RE-)|(RE))', scn) for scn in scenario_results_ext.index]
scenario_results_ext["RE scenario"] = [m[0] if m is not None else "RE" for m in re_scn_matches]

re_scn_sort = {"RE-": 0, "RE": 1, "WIND+": 2, "PV+":3, "RE++": 4}
scenario_results_ext["RE scenario order"] = scenario_results_ext["RE scenario"].replace(re_scn_sort)
scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "RES share [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["RES share [%]"],
            x=xlabels,
            name="RES share",
            marker_color=colors[0],
            hovertemplate='%{y:.2f}%',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext["LCOE [EUR/MWh]"],
                             x=xlabels,
                             mode='markers',
                             #opacity=0.7,
                             marker=dict(color=colors[20], symbol='diamond-wide'),
                             name="LCOE",
                         hovertemplate='%{y:.2f} €/MWh',
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext["LCOH [EUR/MWh]"],
                             x=xlabels,
                            mode='markers',
                             #opacity=0.7,
                             marker=dict(color=colors[20],
                                         symbol='diamond-tall',),
                             name="LCOH",
                         hovertemplate='%{y:.2f} €/MWh',
                             ), secondary_y=True)


fig.update_layout(title=dict(text='Integration of RE and LCOE/H'),
                  yaxis=dict(
                      title="RES share in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  width=900,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="LCOE/LCOH in EUR/MWh", secondary_y=True)
fig.update_xaxes(tickmode = 'linear',)

# 7 Power Grid <a class="anchor" id="7_power_grid"></a>
The following plot shows the distribution of the maximum line loading of all lines in the ABW-Region for each scenario. Each timestep, only one line with the maximum value is represented. Therefore this plot only shows the worst-case line per timestep and neglects the remaining lines. Thus it can only be an indicator for high line loadings and give a vague overview about the grid situation.

In [ ]:
df_line_loading = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Line loading'].max(level='timestamp')*100
                       for scn in scenarios})#.rename(columns=PRINT_NAMES)
df_line_loading = df_line_loading.loc[:,scenario_order]
fig = go.Figure()
for i, (group, data) in enumerate(df_line_loading.iteritems()):
    fig.add_trace(go.Box(
        y=data.values,
        name=group,
        boxmean=True,
        boxpoints=False,
        marker_color=set_colors(39)[1][i],
        showlegend=False))

#fig.update_traces(orientation='h')#, side='positive', width=2, points=False)
fig.update_yaxes(title='%',showspikes=True)
fig.update_xaxes(type='category', tickmode = 'linear',)
fig.update_layout(
    title='Max Lines Loading Distribution', height=800,
    yaxis_showgrid=False, yaxis_zeroline=True, hovermode="x unified",
                 xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order),)

fig.show()

# 8 Flexibility <a class="anchor" id="8_flexibility"></a>

<div class="alert alert-block alert-info">
<b>Notes on the Calculation Flexibility Indicator:</b>

- <b>(1) Annual Autarky (ABW)</b>: degree of autark electricity supply for ABW disregarding dimension of time
  $$Autarky_{Annual,ABW,\%} = \frac{\sum_{t=1}^{8760} E_{supply,mun,t}}{\sum_{t=1}^{8760} E_{demand,mun,t}} \cdot 100\,\%$$
    
    
- <b>(2) Autark Hours (ABW)</b>:
  $$Autark\,hours_{Annual,ABW,\%} = \frac{\sum_{t=1}^{8760} (\frac{E_{supply,ABW,t}}{E_{demand,ABW,t}} \geq 1)}{8760}   \cdot 100\,\%$$
    
    
    
- <b>(3) Utilization Rate (Storage):</b>
    
The Ratio of $Total Cycles_{technology}$ to $Max Cycles_{technology}$
    
$$Total Cycles_{technology}=\frac{E_{tech, discharge}}{C_{technology}}$$
    
$$Max Cycles_{technology}=\frac{1}{2} \cdot timesteps \cdot C_{rate}$$
    
with
    
$$C_{rate}  = \begin{cases}
\frac{P_{n, discharge}}{C_{technology}} & \frac{P_{n, discharge}}{C_{technology}} \leq 1 \\
1 & \, 
\text{otherwise}
\end{cases}$$
    
$$Utilization Rate_{technology} = \frac{Total Cycles_{technology}}{Max Cycles_{technology}}$$
    
- <b>(4) Utilization Rate (DSM):</b>
   
The Ratio of Net activation to DSM Capacity
    
$$Utilization Rate_{DSM} = \frac{DSM \, Net \, Activation}{DSM \, Capacity}$$
    
</div> 

The following plot shows the percentage of autark hours a year in relation to the utilization rate of multiple flexibility options. 

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "Autark hours [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["Autark hours [%]"],
            x=xlabels,
            name="Autark hours in %",
           marker_color=colors[0],
            hovertemplate='%{y:.2f} %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [%]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_color="grey",
                            marker_symbol='diamond-wide',
                            hovertemplate='%{y:.2f} %',
                            name="Battery storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [%]'],
                            x=xlabels,
                            mode='markers',
                            marker_color="indianred",
                            marker_symbol='star-diamond',
                            #opacity=0.7,
                            hovertemplate='%{y:.2f} %',
                            name="Heat storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['DSM Utilization Rate [%]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_symbol='star-square',
                            marker_color="darkorange",
                            hovertemplate='%{y:.2f} %',
                            name="DSM utilization in %",
                             ), secondary_y=True)

fig.update_layout(title=dict(text='Autarky, Utilization of Storage and DSM'),
                  yaxis=dict(
                      title="Autark hours in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=800,
                  width=900,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="Utilization in %", secondary_y=True)
fig.update_xaxes(tickmode = 'linear',)

The following plot shows the percentage of autark hours a year in relation to amount of shifted energy of multiple flexibility  options. 

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "Autark hours [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["Autark hours [%]"],
            x=xlabels,
            name="Autark hours in %",
            marker_color=colors[0],
            hovertemplate='%{y:.2f} %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [GWh]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_color="grey",
                            marker_symbol='diamond-wide',
                            hovertemplate='%{y:.2f} GWh',
                            name="Shifted electricity battery storage in GWh",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [GWh]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_symbol='star-diamond',
                            marker_color="indianred",
                            hovertemplate='%{y:.2f} GWh',
                            name="Shifted electricity heat storage in GWh",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Net DSM activation [GWh]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_symbol='star-square',
                            marker_color="darkorange",
                            hovertemplate='%{y:.2f} GWh',
                            name="Shifted electricity DSM in GWh",
                             ), secondary_y=True)


fig.update_layout(title=dict(text='Autarky, Amount of Storage and DSM', y=.97),
                  yaxis=dict(
                      title="Autark hours in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )
fig.update_yaxes(title_text="Shifted electricity in GWh", secondary_y=True)
fig.update_xaxes(tickmode = 'linear',)

The following plot shows the share of renawbles in relation to the utilization rate of multiple flexibility options. 

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "RES share [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["RES share [%]"],
            x=xlabels,
            name="RES share in %",
            marker_color=colors[0],
            hovertemplate='%{y:.2f} %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [%]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_color="grey",
                            marker_symbol='diamond-wide',
                            name="Battery storage utilization in %",
                            hovertemplate='%{y:.2f} %',
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [%]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_color="indianred",
                            marker_symbol='star-diamond',
                            name="Heat storage utilization in %",
                            hovertemplate='%{y:.2f} %',
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['DSM Utilization Rate [%]'],
                            x=xlabels,
                            mode='markers',
                            #opacity=0.7,
                            marker_symbol='star-square',
                            marker_color="darkorange",
                            name="DSM utilization in %",
                            hovertemplate='%{y:.2f} %',
                             ), secondary_y=True)


fig.update_layout(title=dict(text='RE Share, Utilization of Storage and DSM', y=.97),
                  yaxis=dict(title="RE share in %", titlefont_size=16, tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="Utilization in %", secondary_y=True)
fig.update_xaxes(tickmode = 'linear',)